## Helper Functions

To arrive at the login_page() function, several helper functions are necessary. They are illustrated and tested below.

In [8]:
import hashlib 

#hash the password for increased data security (Input: String, Output: Hash string)
def hash_passwords(password):
    password_bytes = password.encode('utf-8')
    return hashlib.sha256(password_bytes).hexdigest()

password_test = "xxx"
print(hash_passwords(password_test))

cd2eb0837c9b4c962c22d2ff8b5441b7b45805887f051d39bf133b583baf6860


In [9]:
#create and load values of a database for user credentials using a pickle file
import pickle
import os
user_database = "data/user_database.pkl"

def load_user_data(): #(Input: user_database file as pickle, Output: python dictionary with user credentials)
    # Return empty dictionary with "usernames" key if the database is empty or the file is non-existent
    if not os.path.exists(user_database) or os.path.getsize(user_database) == 0:
        return {"username": {}}

    with open(user_database, "rb") as file:
        return pickle.load(file)
    
print(load_user_data())

{'username': {}}


In [ ]:
import pickle 
user_database = "user_database.pkl"

#store username and hashed password in a pickle file (Input: dictionary with user data)
def save_user_data(data):
    with open(user_database, "wb") as file:
        pickle.dump(data, file)

test_data = {'nhollnagel': {hash_passwords(password_test)}}
save_user_data(test_data)

#Result: Data is added to the test pickle file

In [17]:
#authentificator function (inputs: username and passoword based on user input in login_page(), user_data as dictionary, Output: booleans)
def authenticate_user(username, password, user_data):
    if username not in user_data["username"]:
        return False

    #access hashed password assigned to user based on username
    stored_hash = user_data["username"][username]["hashed_password"]

    #call hash_passwords function to access input password provided by user
    input_hash = hash_passwords(password)

    #compare stored and input password by value and return a boolean
    return stored_hash == input_hash

#Test conducted on while entering credentials on the streamlit dashboard

## Main Functions
Displayed below is the intial version as a seperate streamlit page. The session state functionality was later used to integrate the functionality in the final dashbaord. In the final version of the dashboard, the messages throw at the user after login attempts are replaced by setting a session state as boolean. This boolean is the user in the check_login_status() function to either grant access to the desired page or deny it.

In [ ]:
#Combine all functions within the login page

import streamlit as st

def login_page():

    #Configure streamlit page
    st.set_page_config(page_title="Crowd Data Line Graph", page_icon="📈", layout="wide")
    st.title("Login Page")
    st.caption("Welcome to the Sail 2025 Crowd Management Dashboard")

    #offer login or signup option to user
    option = st.selectbox("Login/Signup", ["Login", "Sign Up"])

    #Login functionality

    user_data = load_user_data() #define user data --> python dictionary with key "usernames" and "hashed_password"

    if option == "Login": #if login is selcted...
        st.subheader("Login to your Account")
        login_username = st.text_input("Username")
        login_password = st.text_input("Password", type = "password")

        if st.button("Login"): #if login button is pressed call authenticate_user function

            authentication_state = authenticate_user(login_username, login_password, user_data)

            if authentication_state == True:
                st.success(f"Welcome back {login_username} !")
            
            else: #In case username is not found in database
                st.error("Invalid Username or Password.")


    else: #if sign up is selceted...
        st.subheader("Create new account")
        new_username = st.text_input("Username")
        new_password = st.text_input("Password", type = "password")

        
        if st.button("Create Account"): #if create account is selected...
            #error if not all fields are filled
            if not new_username or not new_password:
                st.warning("Please enter both username and password!")
            #warning in case username already exists
            elif new_username in user_data["usernames"]:
                st.warning("Username already exists. Please login instead.")
            else:
                #call hash_password function to hash password
                hash_pwd = hash_passwords(new_password)
                #add username and hashed password in dictionary format
                user_data["username"][new_username] = {
                "hashed_password": hash_pwd
                }
                #add new user data to pickle file calling the save_user_data function
                save_user_data(user_data)
                st.success("Your account has successfully been created! You can now Log in :)")

    

The security function below allows the correct flow of code. It is called on each page of the dashboard before any data is loaded to check login status of the user.

In [ ]:
def check_login_status():
    #Display of Logout Button on every page of the dashbaord
    st.sidebar.button("Logout", on_click=lambda: st.session_state.update(logged_in=False, username=None), key="logout_sidebar")
    #A lambda function is used as it is single purpose for the creation of a logout button.
    
    #Initialise session state
    if 'logged_in' not in st.session_state:
        st.session_state['logged_in'] = False

    #check if user is not logged in --> rest of code (the page in sidebar) is not loaded
    if not st.session_state['logged_in']:
        st.error("Access Denied. Please log in to view this page. Click '''home''' to log in.")
        st.stop() #stops further execution of the code